# Home work 3: What movie to watch tonight?

## Libraries

In [ ]:
import random
import time
import os
import urllib
import requests
import csv
import json
import pandas as pd
import math
import heapq
import numpy as np

from numpy import dot
from numpy.linalg import norm
from utils import CleanData
from collections import defaultdict
from bs4 import BeautifulSoup
from utils import CleanData, RemoveDuplicates

## Task 1: Data collection

The final output of this task is tsv files of whole movies that makes our dataset.

For this aim, first of all we download all html pages.
For downloading the pages,we use 3 folders that contain html links for reading the movies' links. Because the times of crawling wikipedia pages are too much, we use a random period between 1 to 5 seconds between two crawling. If the wikipedia website blocks our IP, we consider 20 minutes delay to get out of black list and continue downloading.

The code taht we bring here downloading movie2 folder.

### collector

In [ ]:
#I create a beautifulsoup object

url = 'https://raw.githubusercontent.com/CriMenghini/ADM/master/2019/Homework_3/data/movies2.html'
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')
os.chdir(r'C:\Users\cecco\Desktop\AllMovies')
lst = []
i = 0
for link in soup.find_all('a'):
# with 'href' i take all links

    x = link.get('href')
    lst.append(x)
for v in lst:
    try:
        # i store the response in a file html
        
        url = v
        response = urllib.request.urlopen(url)
        c = "article_" + str(i)
        webContent = response.read()
        f = open(c + '.html', 'wb')
        f.write(webContent)
        f.close
        time.sleep(random.randint(1, 5))
        
        # I repeat the operation if I generate an excpetion timeout I stop 1200 seconds
        
    except requests.exceptions.Timeout as e:
        time.sleep(1200)
        url = v
        response = urllib.request.urlopen(url)
        c = "article_" + str(i)
        webContent = response.read()
        f = open(c + '.html', 'wb')
        f.write(webContent)
        f.close
        time.sleep(random.randint(1, 5))
        
        # if I generate an exception for some other reason I skip the file
        
    except Exception as e:
        continue
    finally:
        i += 1

### Parser

In this part, we extract title, intro, plot and some data from infobox.

The title is the text in the first heading of wikipedia html code.

Intro is first paragraph(s) that ends before "contents". The class of "contents" is "toc". So, we find paragraph(s) before this class.

Plot has a specific class that called "plot". So, we just find the paragraph(s) inside this class.

For gathering data from infobox, we find infobox table. separate headings and contents in 2 lists and by searching in heading list, extract what we want.

At the end, collect all this data in a dictionary and save it as tsv file.

In [ ]:
#for loop to crawl all the html file that we downloaded before

for i in range(29992):
    with open(r'C:\Users\shekoufeh\movies3\article_'+str(i)+'.html' ,encoding ='utf8') as f:
        html = f.read()
    soup = BeautifulSoup(html,'html.parser')
    dict={}
    
#title

    title=(soup.select('.firstHeading'))[0].text
    dict['title']=title.replace("\n"," ").strip() 

#intro
    
    for items in soup.select(".toc"):
        
#the paragraphs before class="toc" are in the intro cluster. so we find and merge them as a intro
        
        intro = [item.text for item in items.find_previous_siblings() if item.name=="p"]
        
#this method find the paragrapghs from end to top of the page. so we reverse them to be in the correct order

    intro.reverse()
    
    intro="".join(intro)
    dict['intro']=intro.replace("\n"," ").strip()   
    
#plot
       
    try:
        plot = []

# find the node with id of "Plot"

        mark = soup.find(id="Plot")
        
# walk through the siblings of the parent (H2) node until we reach the next H2 node

        for elt in mark.parent.nextSiblingGenerator():
            if elt.name == "h2":
                break
            if hasattr(elt, "text"):
                plot.append(elt.text)
        
# convert to text 

        plot="".join(plot)
    except:
        plot=''
    
    finally:
        dict['plot']=plot.replace("\n"," ").strip()
    
#extract infobox

    table=(soup.select(".infobox"))[0]

    output_rowc1 = []
    output_rowc0 = []
#gather each columns of infobox in a separate list 

    for table_row in table.findAll('tr'):
        rowc1 = table_row.findAll('td')
        rowc0 = table_row.findAll('th')
        
        for row in rowc1:
            output_rowc1.append(row.text)
        for row in rowc0:
            output_rowc0.append(row.text)

#add information in infobox to dictionary
            
    #film_name
    dict['film_name']=output_rowc0[0]

##search information in the lists, if they were available, we add them to a dictionary, otherwise, we add "NA" to dictionary  
    
    #director
    a = [output_rowc0.index(i) for i in output_rowc0 if "direct" in i.lower()]
    if len(a)>0:
        dict['director']=output_rowc1[a[0]].replace("\n","").strip() 
    else:
        dict['director']="NA"
    
    #producer
    a = [output_rowc0.index(i) for i in output_rowc0 if "produce" in i.lower()]
    if len(a)>0:
        dict['producer']=output_rowc1[a[0]].replace("\n","").strip()  
    else:
        dict['producer']="NA"
    
    #writer
    a = [output_rowc0.index(i) for i in output_rowc0 if "writ" in i.lower()]
    if len(a)>0:
        dict['written']=output_rowc1[a[0]].replace("\n","").strip()
    else:
        dict['written']="NA"
    
    #starring
    a = [output_rowc0.index(i) for i in output_rowc0 if "star" in i.lower()]
    if len(a)>0:
        dict['starring']=output_rowc1[a[0]].replace("\n","").strip()
    else:
        dict['starring']="NA"
    
    #music
    a = [output_rowc0.index(i) for i in output_rowc0 if "music" in i.lower()]
    if len(a)>0:
        dict['music']=output_rowc1[a[0]].replace("\n","").strip()
    else:
        dict['starring']="NA"
    
    #release date
    a = [output_rowc0.index(i) for i in output_rowc0 if "release" in i.lower()]
    if len(a)>0:
        dict['release date']=output_rowc1[a[0]].replace("\n","").strip()
    else:
        dict['release date']="NA"
    
    #running time
    a = [output_rowc0.index(i) for i in output_rowc0 if "run" in i.lower()]
    if len(a)>0:
        dict['runtime']=output_rowc1[a[0]].replace("\n","").strip()
    else:
        dict['runtime']="NA"
    
    #country
    a = [output_rowc0.index(i) for i in output_rowc0 if "country" in i.lower()]
    if len(a)>0:
        dict['country']=output_rowc1[a[0]].replace("\n","").strip()
    else:
        dict['country']="NA"
    
    #language
    b = [output_rowc0.index(i) for i in output_rowc0 if "language" in i.lower()]
    if len(a)>0:
        dict['language']=output_rowc1[a[0]].replace("\n","").strip()
    else:
        dict['language']="NA"
    
    #budget
    a = [output_rowc0.index(i) for i in output_rowc0 if "budget" in i.lower()]
    if len(a)>0:
        dict['budget']=output_rowc1[a[0]].replace("\n","").strip()
    else:
        dict['budget']="NA"
        
#create tsv files
        
    with open('article_'+str(i)+'tsv','w',encoding='utf8') as f:
        fieldnames = ['title','intro','plot','film_name','director', 'producer', 'written', 'starring', 'music', 'release date', 'runtime', 'country', 'language', 'budget']
        writer = csv.DictWriter(f, fieldnames=fieldnames,dialect="excel-tab")
        writer.writeheader()
        writer.writerow(dict)

## Task 2: Search Engine 

### 2.1. Conjunctive query

Now, we want to create two different Search Engines that, given as input a query, return the movies that match the query.

For the homework it has been requested to create a vocabulary file that associates to each word a term_id. Here we are going to define the function used to create the first inverted_index, which will associate to term_id word the list of tsv documents which contain it

In [ ]:
# with this function I create the vocabulary and the first index file

def index1():
    vocabulary ={}
    inverted_idx=defaultdict(list)
    cnt = 1
    for i in range(29992):
        try:
            db = pd.read_csv(r'C:\Users\cecco\Desktop\PagWeb\article_' + str(i) +'.tsv', sep='\t', encoding = 'utf8')
        except FileNotFoundError as e:
            continue
        # i take only intro and plot
        intro = db['intro']
        plot = db['plot']
        # i clean data and remove duplicates
        for x in intro:
            IntroWords = CleanData(x)
        for y in plot:
            PlotWords = CleanData(y)
        AllWords = RemoveDuplicates(IntroWords,PlotWords)
        # I add the words with their id to the vocabulary
        for token in AllWords:
            if token in vocabulary.keys():
                continue
            else:
                vocabulary.update({token:cnt})
                cnt += 1
        # in the index I add key of the word and articles in which it is found
        for w in AllWords:
            x = vocabulary[w]
            inverted_idx[x].append('article_' + str(i))
    os.chdir(r'C:\Users\cecco\Desktop\PagWeb')
    # I save the vocabulary and the inverted index in two file
    with open("vocabulary.json", "w", encoding = "utf8") as myfile:
        myfile.write(json.dumps(vocabulary))
    with open("inverted_index.json", "w", encoding = "utf8") as myfile:
        myfile.write(json.dumps(inverted_idx))

Given a query the Search Engine is supposed to return a list of documents. The conjunctive queries (AND) must br return each documents that contain all the words in the query.

In [ ]:
def searchEngine_1(query):
    query = str(query)
    query = CleanData(query)
    # I load the vocabulary and the inverted index so as not to have to calculate them at the moment
    inverted_index = json.loads(open("inverted_index.json").read())
    vocabulary = json.loads(open("vocabulary.json").read())
    index = []
    # I get the word id from the vocabulary
    for par in set(query):
        if par in vocabulary.keys():
            index.append(vocabulary[par])
    docs = []
    if len(index) != len(set(query)):
        # not all words are in the dictionary
        return print("There are no results that match your query.")
    else:
        # I get the set of all the articles they have all the words from
        for indices in index:
            docs.append(set(inverted_index[str(indices)]))
        docs = set.intersection(*docs)
        if len(docs) == 0:
            # no film has all the words
            return print('there are no articles that contain all the words.')
        else:
            # creo un db con l'insieme dei film e per ognuno stampo title intro plot e url
            result = pd.DataFrame()
            d_url = json.loads(open("d_url.json").read())
            for doc in docs:
                z = d_url[doc]
                df1 = pd.DataFrame({'link': z}, index=[0])
                df = pd.read_csv(doc + ".tsv", delimiter="\t")
                result2 = pd.concat([df, df1], axis=1)
                result = result.append(result2)
            result.reset_index(drop=True, inplace=True)
            return (result)

### 2.2. Conjunctive query & Ranking score

First of all we need to create and inverted_score_index. This index will associate to each word, a list tuple (document, tfidf), with all the documents containing that word, and another file where I store only idf this will be useful to calculate the tfidf of the words in the query.

In [ ]:
def index2():
    os.chdir(r'C:\Users\cecco\Desktop\PagWeb')
    inverted_index = json.loads(open("inverted_index.json").read())
    vocabulary = json.loads(open("vocabulary.json").read())
    inverted2_idx = defaultdict(list)
    Idf_dict = defaultdict(list)
    for i in range(29992):
        try:
            db = pd.read_csv(r'C:\Users\cecco\Desktop\PagWeb\article_' + str(i) + '.tsv', sep='\t', encoding='utf8')
        except FileNotFoundError as e:
            continue
        intro = db['intro']
        plot = db['plot']
        for x in intro:
            IntroWords = CleanData(x)
        for y in plot:
            PlotWords = CleanData(y)
        Plot_Intro = IntroWords + PlotWords
        
        # I take the length of intro and plot to calculate the tf
        lungh = len(Plot_Intro)
        AllWords = RemoveDuplicates(IntroWords, PlotWords)
        
        # update current vocabulary with the new words found in a document
        tfIdf = {}
        IDF = {}
        for token1 in AllWords:
            
            # I count the number of times the word appears in intro and plot always useful for calculating the tf
            x = Plot_Intro.count(token1)
            id_token = vocabulary[token1]
            doc_par = inverted_index[str(id_token)]
            
            # check in how many articles the word useful to calculate idf appears
            num_doc_par = len(doc_par)
            # calculate the tf
            tf = x / lungh
            # calculate idf
            Idf = 1 + math.log(29992 / num_doc_par)
            IDF[token1] = Idf
            # calculate tfidf
            tfIdf[token1] = tf * Idf

        for ww in AllWords:
            l = tfIdf[ww]
            p = IDF[ww]
            xx = vocabulary[ww]
            inverted2_idx[xx].append(('article_' + str(i), l))
            Idf_dict[xx].append(('article_' + str(i), p))
            
            # I create a file by copying into it a dictionary whose key is the id of the word and as values ​
            # a tuple formed by the document of belonging and its tfidf
            
    with open("inverted2_index.json", "w", encoding="utf8") as myfile:
        myfile.write(json.dumps(inverted2_idx))
        
        # I create a file by copying into it a dictionary whose key is the id of the word and as values ​
        # a tuple formed by the document of belonging and its idf this will be useful to calculate the tfidf of the words in the query
    
    with open("Idf_dict.json", "w", encoding="utf8") as myfile:
        myfile.write(json.dumps(Idf_dict))


Now that we have the scored index we can define the functions for the execution of the query. In this section, Cosine Similarity is used to see how much the query is equal to the document found.

To calculate the Cosine Similarity between each document and the query we need to be able to compute the dot product between 2 list of floats and the norm of a list of floats.

In [ ]:
def searchEngine_2(query):
    query = CleanData(query)
    inverted_index = json.loads(open("inverted_index.json").read())
    inverted2_index = json.loads(open("inverted2_index.json").read())
    vocabulary = json.loads(open("vocabulary.json").read())
    Idf_dict = json.loads(open("Idf_dict.json").read())
    index = []
    
    # I calculate the tf of the word in the query since the query is a set is always given by 1 fraction the length of the set
    query_words_tf = 1/len(query)
    query = set(query)
    
    # this part is identical to the search engine 1
    for par in query:
        if par in vocabulary.keys():
            index.append(vocabulary[par])
    docs = []
    if len(index) != len(query):
        return print("There are no results that match your query.")
    else:
        for indices in index:
            docs.append(set(inverted_index[str(indices)]))
        docs = set.intersection(*docs)
        if len(docs) == 0:
            return print('there are no articles that contain all the words.')
        else:
            lst2 = []
           
        # for each document
            for d in docs:
                lst = []
                lst1 = []
               
            # for word in the document i calculate the cosine similarity between words in the document and words in the query
                for i in index:
                    c = inverted2_index[str(i)]
                    for val in c:
                        if val[0] == d:
                            lst.append(val[1])
                    b = Idf_dict[str(i)]
                    for val1 in b:
                        if val1[0] == d:
                            lst1.append(val1[1]*query_words_tf)
                cosine  = [d,round(dot(lst, lst1) / (norm(lst) * norm(lst1)),3)]
                lst2.append(cosine)
               
            # I create a maxheap where I keep documents with relative similarity
            q = [(x[1], x[0]) for x in lst2]
            heapq._heapify_max(q)
            k = 5
            lst4 = []
            for i in range(k):
                    try:
                        c = heapq._heappop_max(q)
                        lst4.append(c)
                    except IndexError:
                        break
            result = pd.DataFrame()
           
        # I create the db to be returned with the first 5 elements with greater similarity
            d_url = json.loads(open("d_url.json").read())
            for tupla in lst4:
                z = d_url[tupla[1]]
                df2 = pd.DataFrame({'similarity' : tupla[0]},index=[0])
                df1 = pd.DataFrame({'link': z}, index=[0])
                df = pd.read_csv(tupla[1] + ".tsv", delimiter="\t")
                result1 = df[['title', 'intro', 'plot']]
                result2 = pd.concat([result1, df1,df2], axis=1)
                result = result.append(result2)
            result.reset_index(drop=True, inplace=True)
            print(result)

## In this part it is possible to choose the search engine to use and the query to be introduced

In [ ]:
# I have the user choose which dimilarity to use and the query to insert
n = input("Enter query number: ")
if n == str(1):
    query = input("Please, write a query: ")
    df = searchEngine_1(query)
    if df is None:
        pass
    else:
        print(df[['title', 'intro', 'plot', 'link']])
if n == str(2):
    query = input("Please, write a query: ")
    searchEngine_2(query)

## Task 3: Define a new score

In this part, we should rank the output of part 2.1.
For defining a function to rank the list of movies, we can use any variables except into and plot.

We decided to ask the user 4 questions for ranking the list of movies and give a weight to each question. 4 variables that asked from user is film_name, director, starring and language. 
The most important variable is film_name. So, we consider 2 type of weights for variables in the condition of answering this question or not.

Another hypothesis is that the default answer for language question is "English". So, if the user don't answer to the language question or any questions, we could have a weighted array for calculating scores that leads us to new ranking.

How the scoring function works?

The function consider a  1*4 array with zero elements for each movie of dataframe. If the entries of user for each question be found in the dataframe, the amount of array change from zero into 1.

In [ ]:
# The input of the function has to be the result from the step 2.1 that contains all the columns

def searchEngine_3(df):  # when we have the film name
    w1 = [0.8, 0.1, 0.05, 0.05]
    # when we don't have the film name
    w2 = [0.0, 0.5, 0.2, 0.3]
    # Requesting inputs from user.
    Film_name = input("If you are looking for special movie enter the film name otherwise enter space:\n ")
    Starring = input("If you are looking for actor enter the actor name otherwise enter space:\n ")
    Director = input("If you are looking for special movie Director enter the Director name otherwise enter space:\n ")
    language = input(" what is your preferred language:? \n ")
    if language == '':
        language = 'English'
    for i in range(len(df)):
        s = [0, 0, 0, 0]
        try:
            if Film_name != '':
                if df.loc[i, 'film_name'].lower().find(Film_name.lower()) != -1:
                    s[0] = 1
        except:
            continue
        try:
            if Starring != '':
                if df.loc[i, 'starring'].lower().find(Starring.lower()) != -1:
                    s[1] = 1
        except:
            continue
        try:
            if Director != '':
                if df.loc[i, 'director'].lower().find(Director.lower()) != -1:
                    s[2] = 1
        except:
            continue
        try:
            if language != '':
                if df.loc[i, 'language'].lower().find(language.lower()) != -1:
                    s[3] = 1
        except:
            continue
        finally:
            if Film_name == '':
                DotProduct = np.dot(s, w2)
                df.loc[i, 'Score'] = DotProduct
            else:
                DotProduct = np.dot(s, w1)
                df.loc[i, 'Score'] = DotProduct
    return (df)

When the function calculates the score of movies by producting this array to weight array, we have a new ranking by sorting the dataframe based on new scores.

In [ ]:
#if the output of search engine_1 is res_searchEngine1 and contains all the informations in infobox

    if res_searchEngine1 is None:
        pass
    else:
        df = searchEngine_3(res_searchEngine1)
        result = df.sort_values(by = ['Score'],ascending=False)
        result.reset_index(drop=True, inplace=True)
        print(result)

## Task 4: Algorithmic question

The function take in input one value the the string of which we must calculate the palindrome. To calculate the palindrome I use the dynamic programming employing time n^2.

In [ ]:
#calculating the length of the maximum palindrome
#is how to calculate the length of the maximum 
#sub-sequence between the input string and its inverse
def Palindroma(X): 
    # calculation Y as the inverse of X
    Y = X[::-1]
    # calculate the length of the string X to then create a matrix where to save the values
    n = len(X) 
    Matrix = [[None]*(n + 1) for i in range(n + 1)]

    for i in range(n + 1): 
        for j in range(n + 1): 
            if i == 0 or j == 0 : 
                Matrix[i][j] = 0
            elif X[i-1] == Y[j-1]: 
                Matrix[i][j] = Matrix[i-1][j-1]+1
            else: 
                Matrix[i][j] = max(Matrix[i-1][j], Matrix[i][j-1])
  
    # Matrix[n][n] contains the length of Palindroma of X[0..n-1] & Y[0..n-1]
    return Matrix[n][n]
Palindroma('DATAMININGSAPIENZA')